### context와 question의 input size를 maximum에 맞추도록 masking

In [1]:
import numpy as np
import pickle

from tqdm import tqdm

In [77]:
# question, answer, context, label
with open('./babi_preprocessd/train_dataset.pkl', 'rb') as f:
    train = pickle.load(f)
with open('./babi_preprocessd/val_dataset.pkl', 'rb') as f:
    val = pickle.load(f)
with open('./babi_preprocessd/test_dataset.pkl', 'rb') as f:
    test = pickle.load(f)    

In [71]:
with open('./babi_preprocessd/c_word_set.pkl', 'rb') as f:
    c_word_set = pickle.load(f)
with open('./babi_preprocessd/q_word_set.pkl', 'rb') as f:
    q_word_set = pickle.load(f)
with open('./babi_preprocessd/a_word_set.pkl', 'rb') as f:
    a_word_set = pickle.load(f)
with open('./babi_preprocessd/cqa_word_set.pkl', 'rb') as f:
    cqa_word_set = pickle.load(f)

In [78]:
print("context word set #: {}".format(len(c_word_set)))
print("question word set #: {}".format(len(q_word_set)))

context word set #: 124
question word set #: 88


In [79]:
mask_index = 0

In [80]:
[train_q, train_a, train_c, train_l] = train
[val_q, val_a, val_c, val_l] = val
[test_q, test_a, test_c, test_l] = test

In [81]:
c_max_sent_len = 0
for tc in train_c:
    for context in tc:
        if len(context) > c_max_sent_len:
            c_max_sent_len = len(context)

In [82]:
print("maximum words in one context sentence: {}".format(c_max_sent_len))

maximum words in one context sentence: 12


In [9]:
q_max_sent_len = 0
for question in train_q:
    if len(question) > q_max_sent_len:
        q_max_sent_len = len(question)

In [10]:
print("maximum words in one question sentence: {}".format(q_max_sent_len))

maximum words in one question sentence: 12


In [99]:
train_c_masked = []
train_q_masked = []
train_l_masked = []
train_c_real_len = []
train_q_real_len = []
# cs: one context
for cs, l, q in tqdm(zip(train_c, train_l, train_q)):
    context_masked = []
    context_real_length = []
    # cs: many sentences
    for context in cs:
        context_real_length.append(len(context))
        diff = c_max_sent_len - len(context)
        if (diff > 0):
            context_mask = np.append(context, [mask_index]*diff, axis=0)
            context_masked.append(context_mask.tolist())
        else:
            context_masked.append(context)
    diff_c = 20 - len(cs)
    context_masked.extend([[0]*12]*diff_c)
    train_c_masked.append(context_masked)

    diff_q = q_max_sent_len - len(q)
    train_q_real_len.append(len(q))
    question_masked = np.array(np.append(q, [mask_index]*diff_q, axis=0))
    train_q_masked.append(question_masked.tolist())
    
    diff_l = 20 - len(l)
    label_masked = np.append(l, np.zeros((diff_l, 20)), axis= 0)
    train_l_masked.append(label_masked.tolist())
    context_real_length.extend([0]*diff_l)
    train_c_real_len.append(context_real_length)

179909it [00:20, 8574.10it/s] 


In [102]:
val_c_masked = []
val_q_masked = []
val_l_masked = []
val_c_real_len = []
val_q_real_len = []
for cs, l, q in tqdm(zip(val_c, val_l, val_q)):
    context_masked = []
    context_real_length = []
    for context in cs:
        context_real_length.append(len(context))
        diff = c_max_sent_len - len(context)
        if (diff > 0):
            context_mask = np.append(context, [mask_index]*diff, axis=0)
            context_masked.append(context_mask.tolist())
        else:
            context_masked.append(context)
    diff_c = 20 - len(cs)
    context_masked.extend([[0]*12]*diff_c)
    val_c_masked.append(context_masked)

    diff_q = q_max_sent_len - len(q)
    val_q_real_len.append(len(q))
    question_masked = np.array(np.append(q, [mask_index]*diff_q, axis=0))
    val_q_masked.append(question_masked.tolist())
    
    diff_l = 20 - len(l)
    label_masked = np.append(l, np.zeros((diff_l, 20)), axis= 0)
    val_l_masked.append(label_masked.tolist())
    context_real_length.extend([0]*diff_l)
    val_c_real_len.append(context_real_length)

19907it [00:01, 12501.57it/s]


In [103]:
test_c_masked = []
test_q_masked = []
test_l_masked = []
test_c_real_len = []
test_q_real_len = []
for cs, l, q in tqdm(zip(test_c, test_l, test_q)):
    context_masked = []
    context_real_length = []
    for context in cs:
        context_real_length.append(len(context))
        diff = c_max_sent_len - len(context)
        if (diff > 0):
            context_mask = np.append(context, [mask_index]*diff, axis=0)
            context_masked.append(context_mask.tolist())
        else:
            context_masked.append(context)
    diff_c = 20 - len(cs)
    context_masked.extend([[0]*12]*diff_c)
    test_c_masked.append(context_masked)

    diff_q = q_max_sent_len - len(q)
    test_q_real_len.append(len(q))
    question_masked = np.array(np.append(q, [mask_index]*diff_q, axis=0))
    test_q_masked.append(question_masked.tolist())
    
    diff_l = 20 - len(l)
    label_masked = np.append(l, np.zeros((diff_l, 20)), axis= 0)
    test_l_masked.append(label_masked.tolist())
    context_real_length.extend([0]*diff_l)
    test_c_real_len.append(context_real_length)

19910it [00:01, 12484.10it/s]


In [107]:
cnt = 0
for c,q,l in tqdm(zip(train_c_masked, train_q_masked, train_l_masked)):
    for context in c:
        if (len(context) != 12) | (len(q) != 12) | (len(l) != 20):
            cnt += 1
            
for c,q,l in tqdm(zip(val_c_masked, val_q_masked, val_l_masked)):
    for context in c:
        if (len(context) != 12) | (len(q) != 12) | len(l) != 20:
            cnt += 1
            
for c,q,l in tqdm(zip(test_c_masked, test_q_masked, test_l_masked)):
    for context in c:
        if (len(context) != 12) | (len(q) != 12) | len(l) != 20:
            cnt += 1            

179909it [00:01, 105004.32it/s]
19907it [00:00, 102719.57it/s]
19910it [00:00, 101863.47it/s]


In [108]:
if cnt == 0:
    print("Masking success!")
else:
    print("Masking process error")

Masking success!


In [109]:
train_dataset_masked = (train_q_masked, train_a, train_c_masked, train_l_masked, train_c_real_len, train_q_real_len)
val_dataset_masked = (val_q_masked, val_a, val_c_masked, val_l_masked, val_c_real_len, val_q_real_len)
test_dataset_masked = (test_q_masked, test_a, test_c_masked, test_l_masked, test_c_real_len, test_q_real_len)

In [110]:
with open('./babi_preprocessd/train_dataset_masked.pkl', 'wb') as f:
    pickle.dump(train_dataset_masked, f)
with open('./babi_preprocessd/val_dataset_masked.pkl', 'wb') as f:
    pickle.dump(val_dataset_masked, f)
with open('./babi_preprocessd/test_dataset_masked.pkl', 'wb') as f:
    pickle.dump(test_dataset_masked, f)